In [1]:
# Import libraries
import numpy as np
from pathlib import Path
import rasterio as rio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.crs import CRS
from rasterio.io import MemoryFile
from rasterio.enums import Resampling
import affine
import os
from shapely.geometry import box
from rasterio.coords import BoundingBox
from rasterio.mask import mask as masker
import pystac

# RAM check
import psutil

In [2]:
# RAM check
process = psutil.Process()
print(process.memory_info().rss / 10**6)

117.98528


In [3]:
# input paths
load_base = Path("../data/input_scene_4/vv/vv.tif")
load_mask = Path("../data/input_scene_4/ex_mask/ex_mask.tif")

save_base = Path("../data/input_scene_4/vv/corrected/vv.tif")


In [4]:
# TEST

with rio.open(load_base) as src:
    print(src.meta)

with rio.open(load_mask) as src:
    print(src.meta)

{'driver': 'GTiff', 'dtype': 'float32', 'nodata': None, 'width': 14513, 'height': 11030, 'count': 1, 'crs': CRS.from_epsg(32645), 'transform': Affine(20.0, 0.0, 645963.1025014379,
       0.0, -20.0, 3013959.0940192696)}
{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': 0.0, 'width': 13544, 'height': 14475, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(9.999999999999987e-05, 0.0, 88.532464704,
       0.0, -9.999999999999987e-05, 26.485432)}


In [5]:
# Function to change CRS system
# REPROJECTION CRS WITHOUT SAVING
# KEEPS IN --> RAM <--


def reproject_crs(file_path, target_crs):
    """Function to load tiff file from path
    with desired crs.
    """

    # Open the input GeoTIFF file
    with rio.open(file_path) as src:
        print(type(src))

        # Read metadata
        src_crs = src.crs
        src_transform = src.transform
        src_width = src.width
        src_height = src.height

        # Calculate the transform for reprojecting
        transform, width, height = calculate_default_transform(
            src_crs, target_crs, src_width, src_height, *src.bounds
        )

        # Create options for the output file
        kwargs = src.meta.copy()
        kwargs.update(
            {
                "crs": target_crs,
                "transform": transform,
                "width": width,
                "height": height,
            }
        )

        # Create an in-memory dataset
        memfile = MemoryFile()
        dst = memfile.open(**kwargs)

        # Reproject and write to the in-memory dataset
        reproject(
            source=rio.band(src, 1),
            destination=rio.band(dst, 1),
            src_transform=src_transform,
            src_crs=src_crs,
            dst_transform=transform,
            dst_crs=target_crs,
            resampling=Resampling.nearest,
        )
    return dst

In [6]:
# Loading all files with CRS:32632 as example
target_crs = CRS.from_epsg(32632)
base = reproject_crs(load_base, target_crs)
mask = reproject_crs(load_mask, target_crs)

<class 'rasterio.io.DatasetReader'>
<class 'rasterio.io.DatasetReader'>


In [7]:
# TEST

print(base.meta)
print(mask.meta)

{'driver': 'GTiff', 'dtype': 'float32', 'nodata': None, 'width': 15461, 'height': 17470, 'count': 1, 'crs': CRS.from_epsg(32632), 'transform': Affine(42.7489038895335, 0.0, 9097095.379141487,
       0.0, -42.7489038895335, 8403721.582311189)}
{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': 0.0, 'width': 18741, 'height': 17422, 'count': 1, 'crs': CRS.from_epsg(32632), 'transform': Affine(22.83709757980859, 0.0, 9262139.073830998,
       0.0, -22.83709757980859, 8051503.261560466)}


In [8]:
# TEST

print(base.bounds)
print(mask.bounds)

BoundingBox(left=9097095.379141487, bottom=7656898.231361039, right=9758036.182177564, top=8403721.582311189)
BoundingBox(left=9262139.073830998, bottom=7653635.347525041, right=9690129.119574191, top=8051503.261560466)


In [9]:
# TEST

print(base.res, mask.res)

(42.7489038895335, 42.7489038895335) (22.83709757980859, 22.83709757980859)


In [10]:
# Getting overlap bounding box between files
# So to keep parts of data that are needed in the RAM

left = [base.bounds.left, mask.bounds.left]
bottom = [base.bounds.bottom, mask.bounds.bottom]
right = [base.bounds.right, mask.bounds.right]
top = [base.bounds.top, mask.bounds.top]

overlap_bounds = BoundingBox(
    left=max(left), bottom=max(bottom), right=min(right), top=min(top)
)

In [11]:
# TEST

print(left)
print(bottom)
print(right)
print(top)
print(overlap_bounds)

[9097095.379141487, 9262139.073830998]
[7656898.231361039, 7653635.347525041]
[9758036.182177564, 9690129.119574191]
[8403721.582311189, 8051503.261560466]
BoundingBox(left=9262139.073830998, bottom=7656898.231361039, right=9690129.119574191, top=8051503.261560466)


In [12]:
# Convert bounds to polygon
overlap_polygon = box(*overlap_bounds)
print(overlap_polygon)

POLYGON ((9690129.119574191 7656898.231361039, 9690129.119574191 8051503.261560466, 9262139.073830998 8051503.261560466, 9262139.073830998 7656898.231361039, 9690129.119574191 7656898.231361039))


In [13]:
# Crop numpy arrays

crop_base, base_transform = masker(base, shapes=[overlap_polygon], crop=True)
crop_mask, mask_transform = masker(mask, shapes=[overlap_polygon], crop=True)

In [14]:
# TEST

print(base_transform)
print(mask_transform)

| 42.75, 0.00, 9262106.15|
| 0.00,-42.75, 8051513.36|
| 0.00, 0.00, 1.00|
| 22.84, 0.00, 9262139.07|
| 0.00,-22.84, 8051503.26|
| 0.00, 0.00, 1.00|


In [15]:
# Create MemoryFile() out of crop_img

def mem_file(image, crop_img, transform):
    profile = image.profile.copy()
    profile.update(
        driver="GTiff",
        height=crop_img.shape[1],
        width=crop_img.shape[2],
        transform=transform,
    )

    memfile = MemoryFile()
    cropped_image = memfile.open(**profile)
    cropped_image.write(crop_img)

    return cropped_image

In [16]:
# Create memory file for base and mask

cropped_base = mem_file(base, crop_base, base_transform)
cropped_mask = mem_file(mask, crop_mask, mask_transform)

In [17]:
# TEST

print(cropped_base.meta)
print(cropped_mask.meta)

{'driver': 'GTiff', 'dtype': 'float32', 'nodata': None, 'width': 10013, 'height': 9231, 'count': 1, 'crs': CRS.from_epsg(32632), 'transform': Affine(42.7489038895335, 0.0, 9262106.148155086,
       0.0, -42.7489038895335, 8051513.363165323)}
{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': 0.0, 'width': 18741, 'height': 17280, 'count': 1, 'crs': CRS.from_epsg(32632), 'transform': Affine(22.83709757980859, 0.0, 9262139.073830998,
       0.0, -22.83709757980859, 8051503.261560466)}


In [18]:
# TEST

print(cropped_base.res, cropped_mask.res)

(42.7489038895335, 42.7489038895335) (22.83709757980859, 22.83709757980859)


In [19]:
# TEST

print(cropped_base.bounds)
print(cropped_mask.bounds)

BoundingBox(left=9262106.148155086, bottom=7656898.231361039, right=9690150.922800984, top=8051513.363165323)
BoundingBox(left=9262139.073830998, bottom=7656878.215381374, right=9690129.119574191, top=8051503.261560466)


In [20]:
# Function to change the resolution of files to desired one

def rescale_image(input_file, scale_factor):
    # Read the data from the source file
    src = input_file
    data = src.read(
        out_shape=(
            src.count,
            int(src.height * scale_factor),
            int(src.width * scale_factor),
        ),
        resampling=Resampling.bilinear,
    )

    # Update the metadata
    transform = src.transform * src.transform.scale(
        (src.width / data.shape[-1]), (src.height / data.shape[-2])
    )

    # Update the profile
    profile = src.profile
    profile.update(
        driver="GTiff",
        height=data.shape[1],
        width=data.shape[2],
        transform=transform,
    )

    memfile = MemoryFile()
    scaled_dataset = memfile.open(**profile)
    scaled_dataset.write(data)

    return scaled_dataset, profile 

In [21]:
# Set resolution to standard

RES = cropped_base.res[0]
mask_refactor = cropped_mask.res[0] / RES

print("mask refactoring scale:", mask_refactor)

mask refactoring scale: 0.534214810251543


In [22]:
# Changing mask file resolution to base file resolution

base_scaled = cropped_base
mask_scaled, _ = rescale_image(mask, mask_refactor)

In [23]:
# Profile

print(base_scaled.meta, mask_scaled.meta, sep="\n")

{'driver': 'GTiff', 'dtype': 'float32', 'nodata': None, 'width': 10013, 'height': 9231, 'count': 1, 'crs': CRS.from_epsg(32632), 'transform': Affine(42.7489038895335, 0.0, 9262106.148155086,
       0.0, -42.7489038895335, 8051513.363165323)}
{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': 0.0, 'width': 10011, 'height': 9307, 'count': 1, 'crs': CRS.from_epsg(32632), 'transform': Affine(42.75197739918018, 0.0, 9262139.073830998,
       0.0, -42.749319225897196, 8051503.261560466)}


In [24]:
# Memory usage monitor 

print(process.memory_info().rss / 10**6)

2888.024064


In [25]:
# USING EXCLUTION MASK FOR BASE DATA

def return_masked_base(base, mask):

    base_data = base.read(1)
    base_meta = base.meta.copy()

    mask_data = mask.read(1)
    mask_meta = mask.meta.copy()

    # Resample mask raster to match primary raster's resolution and transform
    transform, width, height = calculate_default_transform(
        mask.crs, base_meta['crs'], base_meta['width'], base_meta['height'], *mask.bounds)

    mask_resampled = np.empty((base_meta['height'], base_meta['width']), dtype=np.uint8)

    reproject(
        source=mask_data,
        destination=mask_resampled,
        src_transform=mask.transform,
        src_crs=mask.crs,
        dst_transform=base_meta['transform'],
        dst_crs=base_meta['crs'],
        resampling=Resampling.nearest,
        )

    # Apply mask to primary raster
    new_base_data = np.where(mask_resampled == 1, base_data, 0)

    # Update metadata
    base_meta.update(dtype='float32', nodata=0)

    return new_base_data, base_meta

In [26]:
# Calculating final base

new_base_data, new_base_meta = return_masked_base(base_scaled, mask_scaled)


In [ ]:
# Saving into file

save_base.parent.mkdir(parents=True, exist_ok=True)

with rio.open(save_base, "w", **new_base_meta) as m:
        m.write(new_base_data, 1)